In [ ]:
library(twang)
library(weights)
library(Matching)
library(tidyverse)

In [ ]:
data_dir <- file.path("..", "data")

In [ ]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

In [ ]:
library(tableone)

In [ ]:
features <- c("age", "gender", "first_careunit", "weight", "elix_score",
              "saps", "sofa", "vent", "vaso", "sedative",
              "icd_chf", "icd_afib", "icd_renal", "icd_liver",
              "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy", "icu_adm_weekday",
              "vs_map_first", "vs_heart_rate_first", "vs_temp_first", "vs_cvp_first",
              "lab_wbc_first", "lab_hemoglobin_first", "lab_platelet_first",
              "lab_sodium_first", "lab_potassium_first", "lab_bicarbonate_first",
              "lab_chloride_first", "lab_bun_first", "lab_lactate_first",
              "lab_creatinine_first", "lab_ph_first", "lab_po2_first", "lab_pco2_first",
              "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag")

In [ ]:
tab <- CreateTableOne(vars = features,
                      argsNormal = list(var.equal = FALSE),
                      strata = "echo",
                      data = full_data,)
capture.output(tab_df <- tab %>% print(smd = TRUE) %>% as.data.frame(stringsAsFactors = FALSE)) %>% invisible

In [ ]:
tab_df

In [ ]:
ipw_svydesign <- readRDS(file.path(data_dir, "ipw_svydesign.rds"))

In [ ]:
tab_wtd <- svyCreateTableOne(vars = features, strata = "echo", data = ipw_svydesign)

In [ ]:
capture.output(tab_wtd_df <- tab_wtd %>% print %>% as.data.frame(stringsAsFactors = FALSE)) %>% invisible

In [ ]:
tab_wtd_df

In [ ]:
ps_icustay_id <- data.table::fread(file.path(data_dir, "ps_icustay_id"), data.table = FALSE) %>% pull(icustay_id)

In [ ]:
ps_df <- full_data %>% filter(icustay_id %in% ps_icustay_id)

In [ ]:
tab_ps <- CreateTableOne(vars = features,
                         strata = "echo",
                         argsNormal = list(var.equal = FALSE),
                         data = ps_df)

In [ ]:
capture.output(tab_ps_df <- tab_ps %>% print(smd = TRUE) %>% as.data.frame(stringsAsFactors = FALSE)) %>% invisible

In [ ]:
tab_ps_df